In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pathlib
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
data_root_orig = './testData'
data_root = pathlib.Path(data_root_orig)
print(data_root)
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
# label_names.remove('.ipynb_checkpoints')
print(label_names)
label_to_index = dict((name, i) for i, name in enumerate(label_names))

testData
['GabyNg', 'LeeSeng', 'XiaoYan']


In [3]:
test_set_df = pd.read_csv('./test_GabyNg_set.csv')
print(len(test_set_df))

21


In [4]:
pixel = 128
batch_size = 128
tdatagen_rescale = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0,
    height_shift_range=0,
    rotation_range=0,
    zoom_range=0,
    # shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')

In [5]:
filename = 'model/full_resnet_model_v2.h5'
restNetModel = load_model(filename)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
test_generator = tdatagen_rescale.flow_from_dataframe(dataframe=test_set_df, x_col="filename",
                                                                y_col="label",
                                                                class_mode="categorical",
                                                                target_size=(pixel, pixel),
                                                                shuffle=False,
                                                                batch_size=128)

Found 21 validated image filenames belonging to 3 classes.


In [7]:
pred = restNetModel.predict(test_generator)

In [8]:
predout = np.argmax(pred, axis=1)
testout = test_set_df['label'].apply(lambda x: label_to_index[x]).values
print('testout:', testout)
print('predout:', predout)

testScores = metrics.accuracy_score(testout, predout)
confusion = metrics.confusion_matrix(testout, predout)

print("Best accuracy (on testing dataset): %.2f%%" % (testScores * 100))
print(metrics.classification_report(testout, predout, labels=[0,1,2],target_names=label_names, digits=4))
print(confusion)

testout: [0 0 0 0 0 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1]
predout: [1 0 1 1 1 2 2 2 2 1 1 1 0 2 1 1 1 1 1 0 1]
Best accuracy (on testing dataset): 57.14%
              precision    recall  f1-score   support

      GabyNg     0.3333    0.2000    0.2500         5
     LeeSeng     0.4615    0.8571    0.6000         7
     XiaoYan     1.0000    0.5556    0.7143         9

    accuracy                         0.5714        21
   macro avg     0.5983    0.5376    0.5214        21
weighted avg     0.6618    0.5714    0.5656        21

[[1 4 0]
 [1 6 0]
 [1 3 5]]
